# Azure AI Search E2E RAG Demo - Bring Data from GCP and AWS 

## Introduction
In this demo, we will show you how to seamlessly integrate data from Google Cloud Platform storage and Amazon Web Services (AWS) S3 with Azure AI Search using the latest AI vectorization techniques.

## Prerequisites
- 🐍 Python 3.9 or higher
- ☁️ GCP Storage Bucket 
- ☁️ AWS S3 Storage Bucket 
- 🔗 Azure OpenAI Service
- 🔗 Azure AI Search Service
- 🔗 Azure AI Studio MaaS Cohere Embed V3 real-time endpoint

## Features Covered
This demo covers the following features:
- ✅ Stored=False
- ✅ Scalar Quantization to int8
- ✅ Reranking w/full precision vectors
- ✅ Oversampling
- ✅ Integrated Vectorization
- ✅ Reranking with Semantic Ranker
- ✅ **[NEW]** OneLake Data Source Connector
- ✅ **[NEW]** Azure AI Studio MaaS Embedding Skill (We'll use Cohere Embed V3)
- ✅ **[NEW]** Azure OpenAI new `text-embedding-3-large` embedding model
- ✅ **[NEW]** Score Threshold Filtering
- ✅ **[NEW]** MaxTextSizeRecall property for Hybrid Search
- ✅ **[NEW]** Hybrid Weighting
- ✅ **[NEW]** RAG using Azure OpenAI Studio On Your Data SDK

Let's get started!

In [32]:
! pip install azure-search-documents --pre --quiet
! pip install openai python-dotenv azure-identity cohere azure-ai-vision-imageanalysis --quiet

In [33]:
import cohere
import openai
import os
import time
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AzureMachineLearningSkill,
    AzureOpenAIEmbeddingSkill,
    AzureOpenAIModelName,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexProjectionMode,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SplitSkill,
    VectorSearch,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorSimilarityThreshold,
    VectorizableTextQuery,
    VectorizedQuery
)
from dotenv import load_dotenv


In [34]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_STUDIO_COHERE_API_KEY = os.getenv("AZURE_AI_STUDIO_COHERE_API_KEY")
AZURE_AI_STUDIO_COHERE_ENDPOINT = os.getenv("AZURE_AI_STUDIO_COHERE_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
INDEX_NAME = "fsunavala-aws-openai"
ONELAKE_CONNECTION_STRING = os.getenv("ONELAKE_CONNECTION_STRING")
ONELAKE_CONTAINER_NAME = os.getenv("ONELAKE_CONTAINER_NAME")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

In [35]:
# User-specified parameter
USE_AAD_FOR_SEARCH = True  

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=SEARCH_SERVICE_API_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)


Using AAD for authentication.


## Connect to OneLake

There is a few steps you need to do to create this. You can follow the tutorial here on how to create a onelake in Microsoft Fabric. https://learn.microsoft.com/fabric/onelake/create-lakehouse-onelake

In this example, I already congifured shortcuts to Google Cloud (GCS) Storage and Amazon Web Services (AWS) S3. You can follow the tutorial here on how to create a Onelake shortcut in Microsoft Fabric: https://learn.microsoft.com/fabric/onelake/create-gcs-shortcut

## Create a OneLake data source in Azure AI Search

In [36]:
def create_or_update_data_source(indexer_client, container_name, connection_string, index_name, query):
    """
    Create or update a data source connection for Azure AI Search.
    """
    try:
        container = SearchIndexerDataContainer(name=container_name, query=query)
        data_source_connection = SearchIndexerDataSourceConnection(
            name=f"{index_name}-onelake",
            type="onelake",
            connection_string=connection_string, # Fabric Workspace ID 
            container=container, # Lakehouse ID
        )
        data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)
        print(f"Data source '{data_source.name}' created or updated successfully.")
    except Exception as e:
        print(f"Failed to create or update data source due to error: {e}")

indexer_client = SearchIndexerClient(SEARCH_SERVICE_ENDPOINT, azure_search_credential)

index_names = ["fsunavala-aws-openai", "fsunavala-aws-cohere", "fsunavala-gcp-openai", "fsunavala-gcp-cohere"]
queries = ["fsunavala-aws-shortcut/pdfs", "fsunavala-aws-shortcut/pdfs", "fsunavala-gcp-shortcut/pdfs", "fsunavala-gcp-shortcut/pdfs"]

for index_name, query in zip(index_names, queries):
    create_or_update_data_source(indexer_client, ONELAKE_CONTAINER_NAME, ONELAKE_CONNECTION_STRING, index_name, query)

Data source 'fsunavala-aws-openai-onelake' created or updated successfully.
Data source 'fsunavala-aws-cohere-onelake' created or updated successfully.
Data source 'fsunavala-gcp-openai-onelake' created or updated successfully.
Data source 'fsunavala-gcp-cohere-onelake' created or updated successfully.


## Create a search index

In [37]:

def create_fields(vector_search_dimensions):
    return [
        SearchField(
            name="parent_id",
            type=SearchFieldDataType.String,
            sortable=True,
            filterable=True,
            facetable=True,
        ),
        SearchField(name="title", type=SearchFieldDataType.String),
        SearchField(
            name="chunk_id",
            type=SearchFieldDataType.String,
            key=True,
            sortable=True,
            filterable=True,
            facetable=True,
            analyzer_name="keyword",
        ),
        SearchField(name="chunk", type=SearchFieldDataType.String),
        SearchField(
            name="vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=vector_search_dimensions,
            vector_search_profile_name="myHnswProfileSQ",
            stored=False
        ),
    ]


def create_vector_search_configuration(vectorizer_name):
    return VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            ),
            ExhaustiveKnnAlgorithmConfiguration(
                name="myExhaustiveKnn",
                parameters=ExhaustiveKnnParameters(
                    metric=VectorSearchAlgorithmMetric.COSINE
                ),
            ),
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfileSQ",
                algorithm_configuration_name="myHnsw",
                compression_configuration_name="myScalarQuantization",
                vectorizer=vectorizer_name,
            ),
            VectorSearchProfile(
                name="myExhaustiveKnnProfile",
                algorithm_configuration_name="myExhaustiveKnn",
                vectorizer=vectorizer_name,
            ),
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                name="myOpenAI",
                kind="azureOpenAI",
                azure_open_ai_parameters=AzureOpenAIParameters(
                    resource_uri=AZURE_OPENAI_ENDPOINT,
                    deployment_id=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    api_key=AZURE_OPENAI_API_KEY,
                    model_name=AzureOpenAIModelName.TEXT_EMBEDDING3_LARGE,
                ),
            ),
        ],
        compressions=[
            ScalarQuantizationCompressionConfiguration(
                name="myScalarQuantization",
                rerank_with_original_vectors=True,
                default_oversampling=10,
                parameters=ScalarQuantizationParameters(quantized_data_type="int8"),
            )
        ],
    )


def create_semantic_search_configuration():
    return SemanticSearch(
        configurations=[
            SemanticConfiguration(
                name="mySemanticConfig",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="chunk")]
                ),
            )
        ]
    )


def create_search_index(index_name, fields, vector_search, semantic_search):
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search,
    )
    index_client.create_or_update_index(index)


index_client = SearchIndexClient(
    endpoint=SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)

index_names = [
    "fsunavala-aws-openai",
    "fsunavala-aws-cohere",
    "fsunavala-gcp-openai",
    "fsunavala-gcp-cohere",
]

for index_name in index_names:
    vector_search_dimensions = 1024 if "cohere" in index_name else 3072 
    fields = create_fields(vector_search_dimensions)
    vectorizer_name = "myOpenAI" if "openai" in index_name else None
    vector_search = create_vector_search_configuration(vectorizer_name)
    semantic_search = create_semantic_search_configuration()
    create_search_index(index_name, fields, vector_search, semantic_search)
    print(f"Created index: {index_name}")

Created index: fsunavala-aws-openai
Created index: fsunavala-aws-cohere
Created index: fsunavala-gcp-openai
Created index: fsunavala-gcp-cohere


## Create a Skillset    

In [38]:
# Define the skills
def create_split_skill():
    """Creates a split skill to chunk documents into pages."""
    return SplitSkill(
        description="Split skill to chunk documents",
        text_split_mode="pages",
        context="/document",
        maximum_page_length=2000,
        page_overlap_length=500,
        inputs=[InputFieldMappingEntry(name="text", source="/document/content")],
        outputs=[OutputFieldMappingEntry(name="textItems", target_name="pages")],
    )


def create_embedding_skill_cohere(
    azure_ai_studio_cohere_endpoint, azure_ai_studio_cohere_key
):
    """Defines the embedding skill for generating embeddings via AI Studio Cohere."""
    return AzureMachineLearningSkill(
        description="Skill to generate embeddings via AI Studio Cohere",
        context="/document/pages/*",
        scoring_uri=f"{azure_ai_studio_cohere_endpoint}/v1/embed",
        authentication_key=azure_ai_studio_cohere_key,
        inputs=[
            InputFieldMappingEntry(name="texts", source="=[$(/document/pages/*)]"),
            InputFieldMappingEntry(name="input_type", source="='search_document'"),
            InputFieldMappingEntry(
                name="truncate", source="='NONE'"
            ),  # Trim end of input if necessary
            InputFieldMappingEntry(name="embedding_types", source="=['float']"),
        ],
        outputs=[
            OutputFieldMappingEntry(name="embeddings", target_name="aml_vector_object")
        ],
    )


def create_embedding_skill_openai(
    azure_openai_endpoint, azure_openai_embedding_deployment, azure_openai_key
):
    """Defines the embedding skill for generating embeddings via Azure OpenAI."""
    return AzureOpenAIEmbeddingSkill(
        description="Skill to generate embeddings via Azure OpenAI",
        context="/document/pages/*",
        resource_uri=azure_openai_endpoint,
        deployment_id=azure_openai_embedding_deployment,
        api_key=azure_openai_key,
        model_name=AzureOpenAIModelName.TEXT_EMBEDDING3_LARGE,
        dimensions=3072, # Take advantage of the larger model with variable dimension sizes
        inputs=[InputFieldMappingEntry(name="text", source="/document/pages/*")],
        outputs=[OutputFieldMappingEntry(name="embedding", target_name="vector")],
    )


# Define the index projections
def create_index_projections(index_name, is_cohere):
    """Creates index projections for use in a skillset."""
    vector_source = (
        "/document/pages/*/aml_vector_object/float/0"
        if is_cohere
        else "/document/pages/*/vector"
    )
    return SearchIndexerIndexProjections(
        selectors=[
            SearchIndexerIndexProjectionSelector(
                target_index_name=index_name,
                parent_key_field_name="parent_id",
                source_context="/document/pages/*",
                mappings=[
                    InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                    InputFieldMappingEntry(name="vector", source=vector_source),
                    InputFieldMappingEntry(
                        name="title", source="/document/metadata_storage_name"
                    ),
                ],
            ),
        ],
        parameters=SearchIndexerIndexProjectionsParameters(
            projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
        ),
    )


# Create or update the skillset
def create_skillset(client, skillset_name, skills, index_projections):
    """Creates or updates the skillset with embedding and indexing projection skills."""
    skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset to chunk documents and generate embeddings",
        skills=skills,
        index_projections=index_projections,
    )
    try:
        client.create_or_update_skillset(skillset)
        print(f"Skillset '{skillset_name}' created or updated.")
    except Exception as e:
        print(f"Failed to create or update skillset '{skillset_name}': {e}")


split_skill = create_split_skill()

client = SearchIndexerClient(
    SEARCH_SERVICE_ENDPOINT, credential=azure_search_credential
)

index_names = [
    "fsunavala-aws-openai",
    "fsunavala-aws-cohere",
    "fsunavala-gcp-openai",
    "fsunavala-gcp-cohere",
]

for index_name in index_names:
    if "cohere" in index_name:
        embedding_skill = create_embedding_skill_cohere(
            AZURE_AI_STUDIO_COHERE_ENDPOINT, AZURE_AI_STUDIO_COHERE_API_KEY
        )
    else:
        embedding_skill = create_embedding_skill_openai(
            AZURE_OPENAI_ENDPOINT,
            AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            AZURE_OPENAI_API_KEY,
        )
    index_projections = create_index_projections(index_name, "cohere" in index_name)
    skillset_name = f"{index_name}-skillset"
    print(f"Creating skillset: {skillset_name}")
    create_skillset(
        client, skillset_name, [split_skill, embedding_skill], index_projections
    )

Creating skillset: fsunavala-aws-openai-skillset
Skillset 'fsunavala-aws-openai-skillset' created or updated.
Creating skillset: fsunavala-aws-cohere-skillset
Skillset 'fsunavala-aws-cohere-skillset' created or updated.
Creating skillset: fsunavala-gcp-openai-skillset
Skillset 'fsunavala-gcp-openai-skillset' created or updated.
Creating skillset: fsunavala-gcp-cohere-skillset
Skillset 'fsunavala-gcp-cohere-skillset' created or updated.


## Run Indexer

In [39]:
def create_and_run_indexer(index_name, skillset_name, data_source, endpoint, credential):
    indexer_name = f"{index_name}-indexer"
    indexer = SearchIndexer(
        name=indexer_name,
        description="Indexer to index documents and generate embeddings",
        skillset_name=skillset_name,
        target_index_name=index_name,
        data_source_name=data_source,
        field_mappings=[FieldMapping(source_field_name="metadata_storage_name", target_field_name="title")],
    )
    indexer_client = SearchIndexerClient(endpoint, credential)
    indexer_client.create_or_update_indexer(indexer)
    indexer_client.run_indexer(indexer_name)
    print(f"{indexer_name} is created and running.")

index_names = [
    "fsunavala-aws-openai",
    "fsunavala-aws-cohere",
    "fsunavala-gcp-openai",
    "fsunavala-gcp-cohere",
]

for index_name in index_names:
    skillset_name = f"{index_name}-skillset"
    data_source = f"{index_name}-onelake"
    print(f"Creating and running indexer: {index_name}-indexer")
    create_and_run_indexer(index_name, skillset_name, data_source, SEARCH_SERVICE_ENDPOINT, azure_search_credential)
    time.sleep(30)  # Wait for 30 seconds before starting the next indexer

Creating and running indexer: fsunavala-aws-openai-indexer
fsunavala-aws-openai-indexer is created and running.
Creating and running indexer: fsunavala-aws-cohere-indexer
fsunavala-aws-cohere-indexer is created and running.
Creating and running indexer: fsunavala-gcp-openai-indexer
fsunavala-gcp-openai-indexer is created and running.
Creating and running indexer: fsunavala-gcp-cohere-indexer
fsunavala-gcp-cohere-indexer is created and running.


# Search our vector indexes

## Create function for querying Cohere Embed V3 

In [40]:
def generate_cohere_query_embedding(api_key, endpoint, query):
    co = cohere.Client(api_key=api_key, base_url=f"{endpoint}/v1")
    response = co.embed(texts=[query], input_type="search_query")
    return response.embeddings[0]

## Simple vector search (Cohere)

This code performs a simple vector search over the "fsunavala-aws-cohere" index using the Azure AI Studio MaaS Cohere Embed V3 for generating the query embedding. 

In [41]:
query = "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans"

search_client = SearchClient(
    SEARCH_SERVICE_ENDPOINT,
    index_name="fsunavala-aws-cohere",
    credential=azure_search_credential,
)

vector_query = VectorizedQuery(
    vector=generate_cohere_query_embedding(
        AZURE_AI_STUDIO_COHERE_API_KEY, AZURE_AI_STUDIO_COHERE_ENDPOINT, query
    ),
    k_nearest_neighbors=3,
    fields="vector",
)

results = search_client.search(search_text=None, vector_queries=[vector_query])

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['chunk']}")
    print("-" * 50)

Title: Northwind_Standard_Benefits_Details.pdf
Score: 0.83207124
Content: to 

see if there are any generic or over-the-counter options that may be more affordable. 



• Keep track of your allergies, including the type of allergy, the severity of the allergy, the 

medications you are taking, and the results of any allergy tests you have had. This 

information can be helpful for your doctor when making decisions about your care. 

Ambulance 

COVERED SERVICES: Ambulance  

Ambulance services are covered under the Northwind Standard plan, providing you with 

the medical assistance you need in the event of an emergency. When you are in need of an 

ambulance, you can be sure that Northwind Health will cover your transport to the closest 

hospital or medical facility. 

Covered Services:  

The Northwind Standard plan covers ambulance transport to the nearest hospital or 

medical facility in the event of an emergency. This service is available 24 hours a day, seven 

days a week, and

## Simple vector search (OpenAI)
This code performs a simple vector search, retrieving the top 3 nearest neighbors. For the rest of the code, we'll only query the "fsunavala-aws-openai".

In [42]:
search_client = SearchClient(
    SEARCH_SERVICE_ENDPOINT,
    index_name="fsunavala-aws-openai",
    credential=azure_search_credential,
)

query = "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=3, fields="vector")

results = search_client.search(search_text=None, vector_queries=[vector_query])

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['chunk']}")
    print("-" * 50)  

Title: Northwind_Health_Plus_Benefits_Details.pdf
Score: 0.75144523
Content: your local ambulance provider. You should provide them with your 

Northwind Health Plus plan information and Northwind Health will cover the cost of the 

service, minus any applicable coinsurance and copays.  

In conclusion, Northwind Health Plus covers ambulance services when they are medically 

necessary. This includes emergency services and non-emergency services. If you need to 

use an ambulance, it must be one that is in your network and has been approved by 

Northwind Health. You should always contact your primary care physician to determine if a 

service is medically necessary before using an ambulance, as this will help you to avoid any 

out-of-pocket costs. 

Blood Products And Services 

COVERED SERVICES: Blood Products and Services 

Northwind Health Plus covers a variety of blood products and services that are necessary 

for a healthy life. This plan provides coverage for blood tests, tran

## Simple vector search with Score Thresholding

This code performs a simple vector search, applying a score threshold of 0.75 to filter out lower quality matches.

In [43]:
# Define the query
query = "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans"
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="vector",
    threshold=SearchScoreThreshold(value=0.75),
    # threshold=VectorSimilarityThreshold(value=0.75),
)

# Perform the search
results = search_client.search(search_text=None, vector_queries=[vector_query])

# Print the results
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print("-" * 50)  

Title: Northwind_Health_Plus_Benefits_Details.pdf
Score: 0.75144523
--------------------------------------------------


## Hybrid Search with maxTextRecallSize

This code performs a hybrid search, limiting the text recall size to 3 for more limited text/keyword search results, and prints the total count and top results.

In [62]:
# Define the query
query = "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans"
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="vector",
)

# Perform the search
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    hybrid_search=HybridSearch(
        max_text_recall_size=3,
        count_and_facet_mode=HybridCountAndFacetMode.COUNT_RETRIEVABLE_RESULTS,
        # count_and_facet_mode=HybridCountAndFacetMode.COUNT_ALL_RESULTS,
        
    ),
    include_total_count=True,
)

# Print the total count
print(f"Total count: {results.get_count()}")

# Print the results
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print("-" * 50)  

Total count: 4
Title: Northwind_Standard_Benefits_Details.pdf
Score: 0.03279569745063782
--------------------------------------------------
Title: Northwind_Standard_Benefits_Details.pdf
Score: 0.032786883413791656
--------------------------------------------------
Title: Northwind_Health_Plus_Benefits_Details.pdf
Score: 0.01666666753590107
--------------------------------------------------
Title: Northwind_Health_Plus_Benefits_Details.pdf
Score: 0.016129031777381897
--------------------------------------------------


## Hybrid Search Weighting

This code performs a weighted hybrid search, making vector matches 2x more important than keyword matches, and prints the top 3 results.

In [60]:
# Define the query
query = "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans"
vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=3,
    fields="vector",
    weight=2
)

# Perform the search
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    top=3
)

# Print the results
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print("-" * 50)  

Title: Northwind_Standard_Benefits_Details.pdf
Score: 0.049180325120687485
--------------------------------------------------
Title: Northwind_Standard_Benefits_Details.pdf
Score: 0.048924729228019714
--------------------------------------------------
Title: Northwind_Health_Plus_Benefits_Details.pdf
Score: 0.04871794953942299
--------------------------------------------------


## Perform RAG using your data using Azure AI Search and Azure OpenAI Studio
This code leverages Azure OpenAI Studio's Chat On Your Data feature using the OpenAI client and grounding your previously created Azure AI Search vector index as your retriever to get a a generative response from GPT-4.

In [50]:
client = openai.AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model=AZURE_OPENI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    messages=[
        {
            "role": "user",
            "content": "Coverage and exclusions for emergency and non-emergency medical transport under Northwind health plans",
        },
    ],
    extra_body={
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": SEARCH_SERVICE_ENDPOINT,
                    "index_name": "fsunavala-aws-openai",
                    "authentication": {
                        "type": "api_key",
                        "key": SEARCH_SERVICE_API_KEY,
                    },
                    "query_type": "vector_semantic_hybrid",
                    "embedding_dependency": {
                        "type": "deployment_name",
                        "deployment_name": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    },
                    "semantic_configuration": "mySemanticConfig",
                },
            }
        ],
    },
)

import textwrap
if completion.choices:
    message_content = completion.choices[0].message.content
    wrapped_message_content = textwrap.fill(message_content, width=100)
    print(f"AI Assistant (GPT-4): {wrapped_message_content}")

AI Assistant (GPT-4): The Northwind Standard plan covers ambulance services for emergency transport to the nearest
hospital or medical facility, available 24/7, up to the plan's limit. This coverage applies
regardless of whether the ambulance provider is in-network or out-of-network. However, it does not
cover non-emergency transport, such as transport to a doctor's office or for routine medical care
[doc1].   For Northwind Health Plus members, emergency ambulance services are covered without any
out-of-pocket expenses. Additionally, Northwind Health Plus also covers non-emergency ambulance
services when they are deemed medically necessary by a primary care physician. However, if using an
out-of-network ambulance for non-emergency services, members may be responsible for the entire cost
[doc3].   It's important for members to provide their insurance information at the time of transport
and to check with Northwind Health regarding their cost-sharing responsibilities, especially when
tra